# 1. Custom Vision Model Training

In [ ]:
# Setup Custom Vision 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient

endpoint = "https://southcentralus.api.cognitive.microsoft.com"
training_key = "f944c5a7fd3c46ea9f8e8d201385d9cc"
project_name = "CatsVsDogs"
data_dir = 'data'

# Create trainer
trainer = CustomVisionTrainingClient(training_key, endpoint=endpoint)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() 
    if domain.type == "ObjectDetection" and domain.name == "General (compact)")

# Create a new project
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

In [ ]:
from coco_pipe import *

# Get datastream of cats and dogs COCO images
stream = get_coco_stream(
    tags=['cat', 'dog'], 
    ann_file='annotations/instances_val2017.json',
    data_dir=data_dir
) 

# Split datastream into training and validation
train, val = stream | mp.datasplit(split_value=0.3) | mp.make_train_test_split()

# Upload training stream to the Custom Vision service
coco_to_custom_vision(
    stream=train, 
    project_id=project.id, 
    trainer=trainer, 
    data_dir=data_dir
)

In [ ]:
import time

# Train the model
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(5)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

# 2. Models Quantization

 
 ### 2.1 CoreML Quantization

In [7]:
# Compress CoreML model

import coremltools

# Option 1: convert a full precision (float) MLModel to a 16bit quantized MLModel
model_spec = coremltools.utils.load_spec('models/coreml/model.mlmodel')
model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
coremltools.utils.save_spec(model_fp16_spec, 'models/coreml/modelFP16.mlmodel')

# Option 2: convert a full precision (float) MLModel to a 8bit quantized MLModel
model = coremltools.models.MLModel('models/coreml/model.mlmodel')
model_fp8 = coremltools.models.neural_network.quantization_utils.quantize_weights(model, nbits=8)
model_fp8.save('models/coreml/modelFP8.mlmodel')

Quantizing using linear quantization
Optimizing Neural Network before Quantization:
Finished optimizing network. Quantizing neural network..
Quantizing layer layer1/conv
Quantizing layer layer2/conv
Quantizing layer layer3/conv
Quantizing layer layer4/conv
Quantizing layer layer5/conv
Quantizing layer layer6/conv
Quantizing layer layer7/conv
Quantizing layer layer8/conv
Quantizing layer model_outputs0


In [5]:
# Setup CoreML models

from models.coreml.python.predict import *

coreml_16 = coremltools.models.MLModel('models/coreml/modelFP16.mlmodel')
coreml_8 = coremltools.models.MLModel('models/coreml/modelFP8.mlmodel')
coreml_orig = coremltools.models.MLModel('models/coreml/model.mlmodel')

with open('models/coreml/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_coreml_16 = CoreMLObjectDetection(coreml_16, labels)
od_coreml_8 = CoreMLObjectDetection(coreml_8, labels)
od_coreml_orig = CoreMLObjectDetection(coreml_orig, labels)

### 2.2 Tensorflow Quantization
  
 ❗️ **Note:** Don't forget to copy ```models.tflite.python.predict_lite``` file while doing experiments on your own model. This file is not uploaded with the exported models.

In [6]:
# Compress TensorFlow model
import tensorflow as tf

converter = tf.contrib.lite.TFLiteConverter.from_frozen_graph('models/tensorflow/model.pb', ['Placeholder'], ['model_outputs'])
converter.post_training_quantize = True
tflite_model_8 = converter.convert()
open("models/tensorflow/modelFP8.tflite", "wb").write(tflite_model_8)

ConverterError: TOCO failed. See console for info.
/bin/sh: toco_from_protos: command not found




In [8]:
# Setup TensorFlow Original Model

from models.tensorflow.python.predict import TFObjectDetection as OrigTFObjectDetection
from models.tensorflow.python.predict_lite import TFObjectDetection as LiteTFObjectDetection

graph_orig = tf.GraphDef()
with tf.gfile.FastGFile('models/tensorflow/model.pb', 'rb') as f:
    graph_orig.ParseFromString(f.read())
        
# Load labels
with open('models/tensorflow/labels.txt', 'r') as f:
    labels = [l.strip() for l in f.readlines()]

od_tflite_orig = OrigTFObjectDetection(graph_orig, labels)

od_tflite_8 = LiteTFObjectDetection('models/tensorflow/modelFP8.tflite', labels)

Instructions for updating:
Use tf.gfile.GFile.


# 3. Models Validation

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

prediction_key = ''
predictor = CustomVisionPredictionClient(prediction_key, endpoint=endpoint)

In [ ]:
from pipe import Pipe

@Pipe
def apply_quantized_model(stream, data_dir, model, dest_field):
    return (
        stream 
        | mp.apply('filename', dest_field + '_raw', lambda x: model.predict_image(Image.open(join(data_dir, x))))
        | mp.apply([dest_field + '_raw', 'width', 'height'], dest_field, lambda x: x[0]
            | mp.select(lambda p: format_dict(p, x[1], x[2]))
            | mp.as_list
        )
        | mp.delfield([dest_field + '_raw'])
    )  

In [ ]:
from utils import *

val2 = (
    val
    # Predict using online CustomVision
    | mp.apply('filename', 'raw_cv', 
        lambda x: predictor.predict_image(project.id, open(join(data_dir, x), mode="rb"), iteration.id).predictions)
    | mp.apply(['raw_cv', 'width', 'height'], 'cv_predictions', lambda x: x[0]
        | mp.select(lambda p: cv_prediction_as_dict(p, x[1], x[2]))
        | mp.as_list
      )    
    # Predict using CoreML original
    | apply_quantized_model(data_dir, od_coreml_orig, 'coreml_orig_predictions') 
    | mp.as_list
)

In [ ]:
stream[0]

In [11]:
import pickle

pickle.dump(stream, open('results.pickle', 'wb'))
# stream = pickle.load(open('results.pickle', 'rb'))

In [ ]:
# Print reports


for pred_field in [key for key in stream[0].keys() if 'predictions' in key]:
    print("\n===== %s =====\n" % pred_field)
    print_report(
        stream=val, 
        input_tags=input_tags, 
        prob_thresh=0.3, 
        overlap_thresh=0.3,
        pred_field=pred_field, 
        gt_field='ground_truth'
    )